In [ ]:
 !pip install numpy==1.16.1
import numpy as np

In [ ]:
# coding : utf-8

import tensorflow.compat.v1 as tf
tf.executing_eagerly()
#tf.disable_v2_behavior()

# 사용할 모델 라이브러리 import
import sys,os
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np

# 초기 설정
root_dir="./drive/My Drive/kingredient/"
categories=["apple","banana"]
nb_classes=len(categories)
#image_size=50

# 데이터 로딩
def load_dataset():
    x_train, x_test, y_train, y_test=np.load("./drive/My Drive/ByeongNam_test/generate/btest4.npy")
    x_train=x_train.astype("float") / 256
    x_test=x_test.astype("float") / 256
    y_train=np_utils.to_categorical(y_train,nb_classes)
    y_test=np_utils.to_categorical(y_test,nb_classes)
    return x_train, x_test, y_train, y_test

# 모델 구성
def build_model(in_shape):
    model=Sequential()
    model.add(Convolution2D(32,3,3,border_mode='Same', input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Convolution2D(64,3,3,border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64,3,3))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                  metrics=['accuracy'])
    return model

# 모델 학습을 수행하고 저장된 모델을 파일로 저장
def model_train(x,y):
    model=build_model(x.shape[1:])
    model.fit(x,y,batch_size=64,epochs=30) #원래 batch_size=30

    return model

# 모델 평가하기
def model_eval(model,x,y):
    score=model.evaluate(x,y)
    print('loss=',score[0])
    print('accuracy=',score[1])

# 모델 학습 및 평가
x_train, x_test, y_train, y_test=load_dataset()
model=model_train(x_train,y_train)
model_eval(model,x_test,y_test)

# 모델 저장
model.save("./drive/My Drive/remove_back_classify/btest4.h5")



In [ ]:
# h5 → tflite
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model_file("./drive/My Drive/remove_back_classify/remove_back_test1.h5")
tflite_model = converter.convert()
open("remove_back_test11.tflite", "wb").write(tflite_model)
tf.saved_model.save(tflite_model,"./drive/My Drive/remove_back_classify/remove_back_test11.tflite")


In [ ]:
# h5 → pb
from tensorflow import keras
model = keras.models.load_model("./drive/My Drive/remove_back_classify/btest3.h5", compile=False)

export_path = './drive/My Drive/remove_back_pb'
model.save(export_path, save_format="tf")

In [ ]:
# pb → tflite
# convert.py
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
saved_model_dir = './drive/My Drive/remove_back_pb'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()
open('./drive/My Drive/remove_back_classify/btest3.tflite', 'wb').write(tflite_model)

In [ ]:
'''안되는코드'''
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()

model.add(Dense(512, activation='relu', input_dim=784))
#model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

#...

from keras import backend as K

custom_input_tensor = tf.placeholder(tf.float32, shape=(1, 784))
output_tensor = model(custom_input_tensor)

frozen_graph = freeze_session(K.get_session(), output_names[output_tensor.op.name])

tflite_model = tf.contrib.lite.toco_convert(frozen_graph,[custom_input_tensor], [output_tensor])
open("./drive/My Drive/tflite/nmist_mlp.tflite", "wb").write(tflite_model)

In [ ]:
# coding : utf-8
import sys,os
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras.models import load_model
from PIL import Image
import numpy as np

# 테스트 이미지 목록

image_files=["./drive/My Drive/ByeongNam_test/banana_test/banana1.jpeg",
             "./drive/My Drive/ByeongNam_test/banana_test/banana2.jpeg",
             "./drive/My Drive/ByeongNam_test/banana_test/banana3.jpeg",
             "./drive/My Drive/ByeongNam_test/banana_test/banana4.jpeg",
             "./drive/My Drive/ByeongNam_test/banana_test/banana17.jpeg",
             "./drive/My Drive/ByeongNam_test/banana_test/banana18.jpeg",
             "./drive/My Drive/ByeongNam_test/banana_test/banana20.jpg",
             "./drive/My Drive/ByeongNam_test/banana_test/banana20.jpg"

             ]

image_size=50
nb_classes=len(image_files)
categories=["apple","banana"]

X=[]
files=[]
#이미지 불러오기
for fname in image_files:
    img=Image.open(fname)
    img=img.convert("RGB")
    img=img.resize((image_size,image_size))
    in_data=np.asarray(img)
    in_data=in_data.astype("float")/256
    X.append(in_data)
    files.append(fname)

X=np.array(X)

#모델 파일 읽어오기
model=load_model('./drive/My Drive/remove_back_classify/remove_back_test1.h5')

#예측 실행
pre=model.predict(X)

#예측 결과 출력
for i,p in enumerate(pre):
    y=p.argmax()
    print("입력:",files[i])
    print("예측:","[",y,"]", categories[y],"/ Score",p[y])